![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# RAG with LLamaIndex

This notebook uses [LLamaIndex](https://docs.llamaindex.ai/en/stable/) and [Redis](https://redis.com) to setup a basic RAG implementation.

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/RAG/03_llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Environment Setup

### Pull Github Materials
Because you are likely running this notebook in **Google Colab**, we need to first
pull the necessary dataset and materials directly from GitHub.

**If you are running this notebook locally**, FYI you may not need to perform this
step at all.

In [2]:
# NBVAL_SKIP
!git clone https://github.com/redis-developer/redis-ai-resources.git temp_repo
!mv temp_repo/python-recipes/RAG/resources .
!rm -rf temp_repo

Cloning into 'temp_repo'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 138 (delta 68), reused 91 (delta 35), pack-reused 0
Receiving objects: 100% (138/138), 7.19 MiB | 4.45 MiB/s, done.
Resolving deltas: 100% (68/68), done.
mv: rename temp_repo/resources to ./resources: Directory not empty


### Install Python Dependencies

In [2]:
# NBVAL_SKIP
%pip install -U -q llama-index llama-index-vector-stores-redis llama-index-embeddings-cohere llama-index-embeddings-openai

You should consider upgrading via the '/Users/robert.shelton/.pyenv/versions/3.9.12/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


### Install Redis Stack

Later in this tutorial, Redis will be used to store, index, and query vector
embeddings created from PDF document chunks. **We need to make sure we have a Redis
instance available.

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [ ]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [4]:
import os

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

## RAG with LlamaIndex

### Dataset Preparation (PDF Documents)

To best demonstrate Redis as a vector database layer, we will load a single
financial (10k filings) doc and preprocess it using some helpers from LangChain:

- `UnstructuredFileLoader` is not the only document loader type that LangChain provides. Docs: https://python.langchain.com/docs/integrations/document_loaders/unstructured_file
- `RecursiveCharacterTextSplitter` is what we use to create smaller chunks of text from the doc. Docs: https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter

In [16]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.redis import RedisVectorStore

# Load list of pdfs from a folder
data_path = "resources/"
docs = [os.path.join(data_path, file) for file in os.listdir(data_path)]

docs = SimpleDirectoryReader(data_path).load_data()

print(f"Sample doc {docs[0]}")

Sample doc Doc ID: 0cd0b3a7-7b46-4c09-8989-7fd7032e9299
Text: UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington,
D.C. 20549 FORM 10-K (Mark One) ☒ ANNUAL  REPORT PURSUANT T O SECTION
13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year
ended September 24, 2022 or ☐ TRANSITION REPORT PURSUANT T O SECTION
13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the transition
period...


In [8]:
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY")

## Create Index
In the following block, Llama-index will embed the docs provide automatically with OpenAI by default and then store them in the storage_context (Redis).

In [9]:
from llama_index.core import StorageContext
from redis import Redis

redis_client = Redis.from_url(REDIS_URL)

vector_store = RedisVectorStore(redis_client=redis_client, overwrite=True)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(docs, storage_context=storage_context)

14:11:50 redisvl.index.index INFO   Index already exists, overwriting.


### Init retriever and query_engine classes

In [10]:
query_engine = index.as_query_engine()
retriever = index.as_retriever()

### Run vector search
We can see the results of the vector search with the retrieve method

In [14]:
result_nodes = retriever.retrieve("What was nike's revenue in fiscal 23?")
for node in result_nodes:
    print(node)

Node ID: c46402e9-977e-4511-82ff-0c5a925ce960
Text: Table of Contents FISCAL 2023 NIKE BRAND REVENUE HIGHLIGHTS The
following tables present NIKE Brand revenues disaggregated by
reportable operating segment, distribution channel and major product
line: FISCAL 2023 COMPARED TO FISCAL 2022 •NIKE, Inc. Revenues were
$51.2 billion in fiscal 2023, which increased 10% and 16% compared to
fiscal 2022 on...
Score:  0.900

Node ID: 3b568603-3de0-46c4-9a02-266e11122e2f
Text: Table of Contents NORTH AMERICA (Dollars in millions) FISCAL
2023FISCAL 2022 % CHANGE% CHANGE EXCLUDING CURRENCY CHANGESFISCAL 2021
% CHANGE% CHANGE EXCLUDING CURRENCY CHANGES Revenues by: Footwear $
14,897 $ 12,228 22 % 22 %$ 11,644 5 % 5 % Apparel 5,947 5,492 8 % 9 %
5,028 9 % 9 % Equipment 764 633 21 % 21 % 507 25 % 25 % TOTAL REVENUES
$ 21,6...
Score:  0.886



### Run query engine
Now let's get a final RAGlike response

In [15]:
response = query_engine.query("What was nike's revenue in fiscal 23?")
response.response

"NIKE's revenue in fiscal 23 was $51.2 billion."